In [99]:
data = []
with open("input.txt", "r") as f:
    for line in f:
        data.append(line.strip())

# Sort the data so that we have int values first
data = sorted(data)

In [91]:
from dataclasses import dataclass
from enum import Enum
from typing import Optional


class Op(Enum):
    AND = 'AND'
    OR = 'OR'
    LSHIFT = 'LSHIFT'
    RSHIFT = 'RSHIFT'
    NOT = 'NOT'
    ASSIGN = 'ASSIGN'

    def __init__(self, val: str):
        if val == 'AND':
            self = Op.AND
        elif val == 'OR':
            self = Op.OR
        elif val == 'LSHIFT':
            self = Op.LSHIFT
        elif val == 'RSHIFT':
            self = Op.RSHIFT

    def val(self) -> int:
        if self == Op.AND:
            return 10
        elif self == Op.OR:
            return 11
        elif self == Op.ASSIGN:
            return 0
        elif self == Op.NOT:
            return 1
        elif self == Op.LSHIFT:
            return 2
        elif self == Op.RSHIFT:
            return 3
        
@dataclass
class Node:
    name: str
    val: Optional[int]
    instructions: list["Instruction"]

    def __repr__(self):
        return f"Node(name={self.name}, val={self.val}, instructions={self.instructions})"

@dataclass
class Instruction:
    op: Op
    input_1: Node | int
    input_2: Optional[Node | int]
    output_reg: Node
    code: str
    scalar: Optional[int] = None
    times_enqueued: int = 0

    def __lt__(self, rhs):
        return self.op.val() < rhs.op.val()

    def inputs_ready(self):
        if self.op == Op.ASSIGN:
            return self.input_1_val() is not None
        if self.op in [Op.NOT, Op.LSHIFT, Op.RSHIFT]:
            return self.input_1_val() is not None
        else:
            return self.input_1_val() is not None and self.input_2_val() is not None
        
    def input_1_val(self):
        return self.input_1 if type(self.input_1) == int else self.input_1.val

    def input_2_val(self):
        return self.input_2 if type(self.input_2) == int else self.input_2.val
        
    def execute(self) -> int:
        if not self.inputs_ready():
            raise ValueError(f"Inputs not ready for instruction {self}")
        if self.op == Op.ASSIGN:
            return self.input_1_val()
        elif self.op == Op.NOT:
            return (1 << 16) - 1 - self.input_1_val() & 0xFFFF
        elif self.op == Op.LSHIFT:
            return self.input_1_val() << self.scalar & 0xFFFF
        elif self.op == Op.RSHIFT:
            return self.input_1_val() >> self.scalar & 0xFFFF
        elif self.op == Op.AND:
            return self.input_1_val() & self.input_2_val()
        elif self.op == Op.OR:
            return self.input_1_val() | self.input_2_val()
        else:
            raise ValueError(f"Unknown operation {self.op}")

    def __repr__(self):
        return f"Instruction(code={self.code})"

NOTE: Data is sorted so that assignments will come first.

In [95]:
def extract_node_or_value(elem: str, node_map: dict[str, Node]) -> int | Node:
    if elem[0].isdigit():
        return int(elem)
    else:
        return node_map[elem]

def generate_instructions(data: list[str]):
    node_map: dict[str, Node] = {}
    instruction_queue = list[Instruction]()
    # First let's create all of the nodes
    for line in data:
        line_parts = line.split(' ')
        node = Node(name=line_parts[-1], val=None, instructions=[])
        if node.name in node_map:
            raise ValueError(f"Node {node.name} already exists")
        node_map[node.name] = node

    for line in data:
        line_parts = line.split(' ')
        if line_parts[0][0].isdigit() and line_parts[1] == '->':
            # This is an assignment.
            val = int(line_parts[0])
            node = node_map[line_parts[-1]]
            node.val = val
        else:
            if line_parts[0] == 'NOT':
                in_reg = line_parts[1]
                out_reg = line_parts[-1]
                input_1 = extract_node_or_value(in_reg, node_map)
                output = node_map[out_reg]
                instruction = Instruction(op=Op.NOT, input_1=input_1, input_2=None, output_reg=output, code=line)
                input_1.instructions.append(instruction)
            elif line_parts[1] == 'LSHIFT' or line_parts[1] == 'RSHIFT':
                in_reg = line_parts[0]
                shift_amount = int(line_parts[2])
                out_reg = line_parts[-1]
                input_1 = extract_node_or_value(in_reg, node_map)
                output = node_map[out_reg]
                instruction = Instruction(op=Op(line_parts[1]), input_1=input_1, input_2=None, code=line, output_reg=output, scalar=shift_amount)
                if type(input_1) == Node:
                    input_1.instructions.append(instruction)
            elif line_parts[1] == '->':
                # This is a direct assignment
                in_reg = line_parts[0]
                out_reg = line_parts[-1]
                input_1 = extract_node_or_value(in_reg, node_map)
                output = node_map[out_reg]
                instruction = Instruction(op=Op.ASSIGN, input_1=input_1, input_2=None, code=line, output_reg=output)
                if type(input_1) == Node:
                    input_1.instructions.append(instruction)
            else:
                in_reg_1 = line_parts[0]
                in_reg_2 = line_parts[2]
                out_reg = line_parts[-1]
                input_1 = extract_node_or_value(in_reg_1, node_map)
                input_2 = extract_node_or_value(in_reg_2, node_map)
                output = node_map[out_reg]
                instruction = Instruction(op=Op(line_parts[1]), input_1=input_1, input_2=input_2, code=line, output_reg=output)
                if type(input_1) == Node:
                    input_1.instructions.append(instruction)
                if type(input_2) == Node:
                    input_2.instructions.append(instruction)

    for node in node_map.values():
        if node.val is not None:
            instruction_queue.extend(sorted(node.instructions))

    print(len(instruction_queue))
    return (node_map, instruction_queue)

In [100]:
def generate_instructions_and_execute(data: list[str]):
    node_map, instruction_queue = generate_instructions(data)
    while len(instruction_queue):
        instruction = instruction_queue.pop(0)
        if instruction.output_reg.val is not None:
            continue

        if not instruction.inputs_ready():
            instruction.times_enqueued += 1
            instruction_queue.append(instruction)
        else:
            val = instruction.execute()
            instruction.output_reg.val = val
            instruction_queue.extend(sorted([inst for inst in instruction.output_reg.instructions if inst.output_reg.val is None]))

    return node_map

node_map = generate_instructions_and_execute(data)
print(node_map['a'].val)


7
2797


# Test

In [93]:
test_input = [
    '123 -> x',
    '456 -> y',
    'x AND y -> d',
    'x OR y -> e',
    'x LSHIFT 2 -> f',
    'y RSHIFT 2 -> g',
    'NOT x -> h',
    'NOT y -> i',
]

node_map = generate_instructions_and_execute(test_input)
for k, v in node_map.items():
    print(f'{k}: {v.val}')



8
filling in h with 65412 from Instruction(code=NOT x -> h)
filling in f with 492 from Instruction(code=x LSHIFT 2 -> f)
filling in d with 72 from Instruction(code=x AND y -> d)
filling in e with 507 from Instruction(code=x OR y -> e)
filling in i with 65079 from Instruction(code=NOT y -> i)
filling in g with 114 from Instruction(code=y RSHIFT 2 -> g)
x: 123
y: 456
d: 72
e: 507
f: 492
g: 114
h: 65412
i: 65079
